In [63]:
# pip install ortools
import pandas as pd
from scipy.spatial import distance_matrix
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

# Import inspection locations
csv_path = "inspection_locations.csv"  # This is located in the same directory
inspection_locations = pd.read_csv(csv_path, index_col='name')

# Randomly sample 8 inspections to conduct today (this would normally be done by due date)
today_inspections = inspection_locations.sample(8,random_state=123)

# Create distance matrix for today's inspections
distance_matrix =  pd.DataFrame(distance_matrix(today_inspections.loc[:,('lat','lng')].to_numpy(), today_inspections.loc[:,('lat','lng')].to_numpy()),
                                index=today_inspections.index, columns=today_inspections.index)

# Scale up the distance matrix
distance_matrix = distance_matrix*10_000
distance_matrix = distance_matrix.values.tolist()

In [62]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = 1  # Only one vehicle for this problem
    data['depot'] = 0  # The depot is index 0 in the distance matrix
    return data

def print_solution(manager, routing, solution):
    """Prints solution on console."""
#     print('Objective: {} miles'.format(solution.ObjectiveValue()))  # This isn't accurate when I scale up the distance matrix
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

def main():
    data = create_data_model()
    
    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Creates the routing model
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # This defines the cost of travel.  In this case it is just the distance.
    # This could be modified to account for different vehicle types or other costs, such as tolls.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # This search parameter finds the first solution that works
    # PATH_CHEAPEST_ARC just uses the least expensive route available that doesn't lead to a previously visited node
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # # This search parameter takes some extra time to explore outside of any local minima it may encounter
    # search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    # search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    # search_parameters.time_limit.seconds = 30
    # search_parameters.log_search = True

    # Print solution for the optimal route
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(manager, routing, solution)
        
if __name__ == '__main__':
    main()

Route for vehicle 0:
 0 -> 2 -> 4 -> 3 -> 5 -> 7 -> 6 -> 1 -> 0



In [ ]:
# Next Steps:

# Consolidate code it into a nice function so it can be run simply

# After that, I need to connect it to a data source so I can pull in data and assign dummy inspection dates
# I can use the FourSquare API to pull in several dozen locations at once
# Dates can be assigned randomly in groups of 5, and then the optimizer can pick the soonest 5
# Replicating the date 5 times could be hard, and it's totally contrived anyway. As a workaround, just assign a number between 1 and len(list)
# and take the first 5 items.

# Also, need to check and make sure it's not getting stuck in a local minimum
# Might also be nice to use Manhattan distance instead of Euclidean since it's pretty much on a grid system
# Later draft could incorporate Google Maps API for actual driving routes